<a href="https://colab.research.google.com/github/johannssh/LangChain-Vector-Databases-in-Production/blob/main/Articles_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain==0.0.208 openai newspaper3k
!pip install --upgrade openai tiktoken wandb -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00


In [ ]:
import os
import openai
import tiktoken
import wandb
from pprint import pprint
from getpass import getpass
from wandb.integration.openai import autolog

In [ ]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
  openai.api_key = os.getenv("OPENAI_API_KEY", "")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

Paste your OpenAI key from: https://platform.openai.com/account/api-keys
··········
OpenAI API key configured


In [ ]:
# start logging to W&B
autolog({"project":"ArticleSummarizer", "job_type": "Project 1: LangChain & Vector Databases in Production"})

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import requests
from newspaper import Article

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

article_urls = "https://www.nytimes.com/2023/09/29/health/mosquitoes-malaria-disease-climate-change.html"

session = requests.Session()

try:
    response = session.get(article_urls, headers=headers, timeout=10)

    if response.status_code == 200:
        article = Article(article_urls)
        article.download()
        article.parse()

        print(f"Title: {article.title}")
        print(f"Text: {article.text}")

    else:
        print(f"Failed to fetch article at {article_urls}")
except Exception as e:
    print(f"Error occurred while fetching article at {article_urls}: {e}")

Title: Mosquitoes Are a Growing Public Health Threat, Reversing Years of Progress
Text: Along hundreds of miles of Lake Victoria’s shoreline in Kenya, a squadron of young scientists and an army of volunteers are waging an all-out war on a creature that threatens the health of more people than any other on earth: the mosquito.

They are testing new insecticides and ingenious new ways to deliver them. They are peering in windows at night, watching for the mosquitoes that home in on sleeping people. They are collecting blood — from babies, from moto-taxi drivers, from goat herders and from their goats — to track the parasites the mosquitoes carry.

But Eric Ochomo, the entomologist leading this effort on the front lines of global public health, stood recently in the swampy grass, laptop in hand, and acknowledged a grim reality: “It seems as though the mosquitoes are winning.”

Less than a decade ago, it was the humans who appeared to have gained the clear edge in the fight — more than a c

In [ ]:
from langchain.schema import (
    HumanMessage
)

# we get the article data from the scraping part
article_title = article.title
article_text = article.text

# prepare template for prompt
template = """You are a very good assistant that summarizes online articles.

Here's the article you want to summarize.

==================
Title: {article_title}

{article_text}
==================

Write a summary of the previous article.
"""

prompt = template.format(article_title=article.title, article_text=article.text)

messages = [HumanMessage(content=prompt)]

In [ ]:
from langchain.chat_models import ChatOpenAI

# load the model
chat = ChatOpenAI(model_name="gpt-4", temperature=0)

In [ ]:
# generate summary
summary = chat(messages)
print(summary.content)

The mosquito has become a growing public health threat, particularly along the shoreline of Lake Victoria in Kenya, reversing years of progress in controlling the insect. Despite efforts by scientists and volunteers to test new insecticides and innovative delivery methods, the mosquito population continues to thrive. Less than a decade ago, humans seemed to be winning the battle against mosquitoes, but recent years have seen a reversal in this trend.


In [ ]:
# prepare template for prompt
template = """You are an advanced AI assistant that summarizes online articles into bulleted lists.

Here's the article you need to summarize.

==================
Title: {article_title}

{article_text}
==================

Now, provide a summarized version of the article in a bulleted list format.
"""

# format prompt
prompt = template.format(article_title=article.title, article_text=article.text)

# generate summary
summary = chat([HumanMessage(content=prompt)])
print(summary.content)

- Young scientists and volunteers are battling mosquitoes along the shoreline of Lake Victoria in Kenya due to the health threat they pose.
- They are testing new insecticides and innovative delivery methods, observing mosquito behavior, and collecting blood samples to track the parasites mosquitoes carry.
- Eric Ochomo, the entomologist leading the effort, admits that the mosquitoes seem to be winning the battle.
- Less than a decade ago, humans seemed to have the upper hand in the fight against mosquitoes.
- However, in recent years, the progress made has not only stalled but reversed, indicating a growing public health threat.
